In [ ]:
from sklearn.model_selection import LeaveOneOut, cross_val_score
from sklearn.metrics import f1_score
from joblib import load

# Load the breast cancer dataset
X, y = load_breast_cancer(return_X_y=True)

# Load the pre-trained model
model = load('model.joblib')

# Use LOOCV to evaluate the model's performance and find the best threshold for classification
loo = LeaveOneOut()
cv_scores = []
thresholds = []

for train_index, test_index in loo.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Make predictions on the test instance
    y_pred_prob = model.predict_proba(X_test)[:, 1]
    
    # Find the best threshold for classification
    best_threshold = 0
    best_score = 0
    for threshold in np.arange(0, 1.01, 0.01):
        y_pred = (y_pred_prob > threshold).astype(int)
        score = f1_score(y_test, y_pred)
        if score > best_score:
            best_score = score
            best_threshold = threshold
    
    # Store the best threshold and the F1 score for this instance
    thresholds.append(best_threshold)
    cv_scores.append(best_score)

# Calculate the mean and standard deviation of the F1 scores
mean_f1 = np.mean(cv_scores)
std_f1 = np.std(cv_scores)

print("LOOCV F1 score: {:.3f} +/- {:.3f}".format(mean_f1, std_f1))